# Week 4 - Part II: Sequence Prediction with HMMs

In this exercise you will implement the Viterbi algorithm for decoding in sequence tagging. The parameter estimation for the parameters of the model (emission and transition) is provided, as they are similar to parameter estimation with smoothing that you have seen in the Naive Bayes lecture.

### Hidden Markov Models (HMM) for sequence tagging

In this part of the exercise you are going to implement a HMM POS tagger.

First, lets read in the data. We provide some helper functions for it (see `myutils.py`).


In [1]:
import codecs
import numpy as np
import myutils
from collections import defaultdict

# load data
train_data = myutils.read_conll_file("data/da_ddt-ud-train.conllu")
dev_data = myutils.read_conll_file("data/da_ddt-ud-dev.conllu")

In [2]:
for e in dev_data:
    print(e)

RON', 'VERB', 'ADP', 'DET', 'NOUN', 'PUNCT'])
(['Kinesisk', 'nytår', 'er', 'fejret', 'med', 'frimærker', 'af', 'Hongkong', 'og', 'Sydkorea', 'siden', '1967', ',', 'at', 'Republic', 'of', 'China', 'på', 'Taiwan', 'siden', '1969', ',', 'af', 'Japan', 'siden', '1976', ',', 'af', 'Macau', 'siden', '1984', 'og', 'af', 'Kommunist-China', 'siden', '1992', '.'], ['ADJ', 'NOUN', 'AUX', 'VERB', 'ADP', 'NOUN', 'ADP', 'PROPN', 'CCONJ', 'PROPN', 'ADP', 'NUM', 'PUNCT', 'SCONJ', 'PROPN', 'X', 'PROPN', 'ADP', 'PROPN', 'ADP', 'NUM', 'PUNCT', 'ADP', 'PROPN', 'ADP', 'NUM', 'PUNCT', 'ADP', 'PROPN', 'ADP', 'NUM', 'CCONJ', 'ADP', 'PROPN', 'ADP', 'NUM', 'PUNCT'])
(['Værelset', 'skrumpede', '.'], ['NOUN', 'VERB', 'PUNCT'])
(['Her', 'er', 'hun', 'dansk', 'koordinator', 'for', 'et', 'kontaktnet', 'af', 'skoler', 'over', 'hele', 'verden', '.'], ['ADV', 'AUX', 'PRON', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT'])
(['Sidstnævnte', 'er', 'af', 'ikke', 'umiddelbart', 'indlysende

## HMM class
We provide an HMM class, which includes the estimation of the transition as well as the emission probabilities:

In [3]:
class HMM(object):
    def __init__(self):
        """
        initialize model parameters
        :return:
        """
        self.START = '_START_'
        self.UNK = 'UNK'
        self.STOP = '_STOP_'
        self.transitions = defaultdict(lambda: defaultdict(float))
        self.emissions = defaultdict(lambda: defaultdict(float))
        self.vocabulary = set()
        self.tags = set()

    def fit(self, train_data):
        """
        fit model to a file in CoNLL format.
        :param file_name:
        :return:
        """
        counts = defaultdict(int)

        # record all used tags and words
        for (words, tags) in train_data:
            
            # iterate over sentence
            for i, (word, tag) in enumerate(zip(words, tags)):
                self.tags.add(tag)
                self.vocabulary.add(word)
                counts[word] += 1

        ## collect counts 
        for (words, tags) in train_data:
            
            # add stop symbol
            words=words+[self.STOP]
            tags=tags+[self.STOP]

            # iterate over sentence
            for i, (word, tag) in enumerate(zip(words, tags)):

                if i==0:
                    prev_tag=self.START

                    # record only transition from start
                    self.transitions[prev_tag][tag] += 1
                    
                else:
                    prev_tag=tags[i-1]

                    # record count for transition
                    self.transitions[prev_tag][tag] += 1

                    if i < len(words)-1:
                        # record count for emissions
                        if counts[word] < 2:
                            self.emissions[tag][self.UNK] += 1
                        # note that infrequent words are counted twice
                        self.emissions[tag][word] += 1

        ## e(tag|word) = count(t->word)/count(word)
        for tag in self.emissions:
            total_tag=sum(self.emissions[tag].values())
            for word in self.emissions[tag]:
                prob_word_given_tag = self.emissions[tag][word] / float(total_tag)
                self.emissions[tag][word] = prob_word_given_tag

        ## t(tag|prevtag) = count(prevtag,tag)/ count(prevtag)
        for prevtag in self.transitions:
            total_prevtag=sum(self.transitions[prevtag].values())
            for tag in self.transitions[prevtag]:
                prob_tag_given_prevtag = self.transitions[prevtag][tag] / float(total_prevtag)
                self.transitions[prevtag][tag] =  prob_tag_given_prevtag

    def predict(self, data, method='most_likely'):
        """
        predict the most likely tag sequence for all sentences in data

        :param data: a list of sentences
        :param method: viterbi or most likely decoding
        :return: list of predicted tag sequences
        """
        results = []
        for sentence in data:
            if method == 'viterbi':
                results.append(self.predict_viterbi(sentence[0]))
            else:
                results.append(self.predict_most_likely(sentence[0]))
        return results


In the following function you are supposed to return the most likely tag for each word:

In [4]:
def predict_most_likely(self, sentence):
    """
    predict the single most likely tag (from training data) for every token in sentence
    (i.e., just looks at a single tag at a time, no context)
        
    :sentence: list of tokens
    :return: list of tags
    """
    tagSeq = []
    for word in sentence:
        max_score = 0
        best_tag = 'NOUN'
        for tag in self.tags:
            if (len(tagSeq) == 0):
                cs = self.transitions[self.START][tag] * self.emissions[tag][word]
                if cs > max_score:
                    max_score = cs
                    best_tag = tag
            else:
                cs = self.transitions[tagSeq[-1]][tag] * self.emissions[tag][word]
                if cs > max_score:
                    max_score = cs
                    best_tag = tag
        tagSeq.append(best_tag)
    return tagSeq

# To make the function link to the class in the previous cell
HMM.predict_most_likely = predict_most_likely


The following code is used to train a model, and predict using the most_likely strategy

In [5]:
# create new model
hmm = HMM()

# fit model to training data
hmm.fit(train_data)

# get most likely tag predictions 
most_likely_predictions = hmm.predict(dev_data, method='most_likely')

# evaluate
gold = [x[1] for x in dev_data]
sent_level, word_level = myutils.evaluate(gold, most_likely_predictions)

print('most likely scores:')
print('sent level:  {:.4f}'.format(sent_level))
print('word level:  {:.4f}'.format(word_level))
print()

most likely scores:
sent level:  0.1560
word level:  0.8712



In the following function you are supposed to implement the Viterbi algorithm. The step from the START symbol to the first word is already implemented.

In [ ]:
def predict_viterbi(self,sentence):
    """
    predict the most likely tag sequences using the Viterbi algorithm

    :sentence: list of tokens
    :return: list of tags
    """

    # replace unknown words for simplicity
    for i in range(len(sentence)):
        if sentence[i] not in self.vocabulary:
            sentence[i] = self.UNK

    # prepare data structures
    N = len(sentence)
    viterbiProbs = np.zeros((N, len(self.tags)))
    # viterbiBacktrace can be used to remember which previous tag was used
    viterbiBacktrace = np.zeros((N, len(self.tags)), dtype=int)
    # make self.tags a list, so we can use indexes
    self.tags = sorted(self.tags)
    # initialize first step (from START)
    for tagIdx, tag in enumerate(self.tags):
        emisProb = self.emissions[tag][sentence[0]]
        transProb = self.transitions[self.START][tag]
        viterbiProbs[0][tagIdx] = emisProb * transProb
    print(sentence)
    # process the rest of the sentence
    for t in range(1,N):
        #TODO implement
        # find max probability of transition from previous states
        # multiply with emission
        # save both probability and path
        for c_idx, current_tag in enumerate(self.tags):
            max_trans = 0
            max_index = -1
            for p_idx, prev_tag in enumerate(self.tags):
                c_trans = self.transitions[prev_tag][current_tag] * self.emissions[current_tag][sentence[t]] * viterbiProbs[t - 1, p_idx]
                #print('current_tag: {}\t prev_tag: {}\t token: {}\t transition_prob: {}'.format(current_tag, prev_tag, sentence[t], c_trans))
                if c_trans > max_trans:
                    max_trans = c_trans
                    max_index = p_idx
            viterbiProbs[t, c_idx] = max_trans
            viterbiBacktrace[t, c_idx] = max_index
        print(viterbiBacktrace)
        print(viterbiProbs)

    # final step (to STOP)
    #TODO implement
    
    # retrieve the most likely sequence from backtrace
    #TODO implement
    tags = list()
    m = (np.max(viterbiProbs[-1, :]))
    depth = viterbiProbs.shape[0]
    print(depth)
    if m > 0:
        first_step = np.argmax(viterbiProbs[-1,:])
        print(first_step)
    else:
        return ['NOUN' for _ in range(depth)]
    
    tags = []
    step = 
    while depth > 0:
        tags.insert(0, self.tags[])
    
    return NotImplementedError

# To make the function link to the class in a previous cell
HMM.predict_viterbi = predict_viterbi


In the following code-snippet the scores of the Viterbi implementation are evaluated. How much higher is the word-level accuracy? How much higher is the sentence-level accuracy?

In [22]:
# get viterbi predictions
viterbi_predictions = hmm.predict(dev_data[2:4], method='viterbi')
# evaluate
print(viterbi_predictions)
#sent_level, word_level = myutils.evaluate(gold, viterbi_predictions)
#print('viterbi scores:')
#print('sent level:  {:.4f}'.format(sent_level))
#print('word level:  {:.4f}'.format(word_level))
#print()


', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB',

In [42]:
for element in gold:
    print(element)

 'PRON', 'VERB', 'ADP', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'CCONJ', 'PRON', 'PRON', 'ADV', 'ADV', 'VERB', 'PUNCT', 'PRON', 'PRON', 'VERB', 'PART', 'AUX', 'VERB', 'PUNCT', 'SCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'ADV', 'PUNCT', 'CCONJ', 'ADP', 'ADV', 'VERB', 'PRON', 'PART', 'VERB', 'PRON', 'ADV', 'ADP', 'NOUN', 'PUNCT']
['ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'PUNCT', 'AUX', 'NOUN', 'PROPN', 'PROPN', 'NUM', 'ADV', 'VERB', 'DET', 'NOUN', 'ADV', 'PUNCT', 'PRON', 'VERB', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'VERB', 'CCONJ', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'ADP', 'NOUN', 'AUX', 'VERB', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADV', 'ADP', 'NOUN', 'ADP', 'NOUN', 'PUNCT']
['PUNCT', 'PRON', 'AUX', 'DET', 'NOUN', 'PUNCT', 'SCONJ', 'PRON', 'VERB', 'PUNCT', 'PRON', 'VERB', 'PRON', 'ADP', 'PRON', 'PUNCT', 'PROPN', 'PUNCT', 'VERB', 'PROPN', 'DET', 'NOUN', 'PUNCT', 'SCONJ', 'PRON', 'AUX', 'ADV', 'ADP', 'NOUN', 'PUNCT']
['PRON', 'VERB', 'ADV', 'DET', 'A